In [124]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.svm import SVC
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [5]:
cars_train = pd.read_csv("cars_train.csv", header=None)
cars_test = pd.read_csv("cars_test.csv", header=None)

In [10]:
print(cars_train.shape)
cars_train.head()

(1382, 7)


,0,1,2,3,4,5,6
0,vhigh,high,3,more,small,low,unacc
1,low,vhigh,3,4,small,med,unacc
2,low,high,5more,more,big,low,unacc
3,high,med,4,2,small,med,unacc
4,low,low,3,more,big,med,good


In [12]:
print(cars_test.shape)
cars_test.head()

(346, 7)


,0,1,2,3,4,5,6
0,med,vhigh,5more,4,small,low,unacc
1,vhigh,high,2,2,big,med,unacc
2,low,high,2,more,small,low,unacc
3,vhigh,vhigh,3,2,big,high,unacc
4,low,med,4,4,med,med,good


In [13]:
cars_train.columns=['buying','maint','doors','persons','lug_boot','safety','classes']
cars_test.columns=['buying','maint','doors','persons','lug_boot','safety','classes']

In [14]:
cars_train.head()

,buying,maint,doors,persons,lug_boot,safety,classes
0,vhigh,high,3,more,small,low,unacc
1,low,vhigh,3,4,small,med,unacc
2,low,high,5more,more,big,low,unacc
3,high,med,4,2,small,med,unacc
4,low,low,3,more,big,med,good


In [15]:
cars_test.head()

,buying,maint,doors,persons,lug_boot,safety,classes
0,med,vhigh,5more,4,small,low,unacc
1,vhigh,high,2,2,big,med,unacc
2,low,high,2,more,small,low,unacc
3,vhigh,vhigh,3,2,big,high,unacc
4,low,med,4,4,med,med,good


In [16]:
cars_test.describe()

,buying,maint,doors,persons,lug_boot,safety,classes
count,346,346,346,346,346,346,346
unique,4,4,4,3,3,3,4
top,low,low,2,2,med,low,unacc
freq,94,94,92,122,122,118,251


In [18]:
cars_train.describe()

,buying,maint,doors,persons,lug_boot,safety,classes
count,1382,1382,1382,1382,1382,1382,1382
unique,4,4,4,3,3,3,4
top,high,vhigh,5more,more,big,high,unacc
freq,349,352,349,466,467,463,959


In [19]:
cars_train.isna().sum()

buying      0
maint       0
doors       0
persons     0
lug_boot    0
safety      0
classes     0
dtype: int64

In [20]:
cars_test.isna().sum()

buying      0
maint       0
doors       0
persons     0
lug_boot    0
safety      0
classes     0
dtype: int64

In [21]:
cars_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    346 non-null    object
 1   maint     346 non-null    object
 2   doors     346 non-null    object
 3   persons   346 non-null    object
 4   lug_boot  346 non-null    object
 5   safety    346 non-null    object
 6   classes   346 non-null    object
dtypes: object(7)
memory usage: 19.1+ KB


In [22]:
cars_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1382 entries, 0 to 1381
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1382 non-null   object
 1   maint     1382 non-null   object
 2   doors     1382 non-null   object
 3   persons   1382 non-null   object
 4   lug_boot  1382 non-null   object
 5   safety    1382 non-null   object
 6   classes   1382 non-null   object
dtypes: object(7)
memory usage: 75.7+ KB


In [23]:
y_actual = cars_test.drop("classes", axis=1, inplace=True)
cars_test.head()

,buying,maint,doors,persons,lug_boot,safety
0,med,vhigh,5more,4,small,low
1,vhigh,high,2,2,big,med
2,low,high,2,more,small,low
3,vhigh,vhigh,3,2,big,high
4,low,med,4,4,med,med


In [25]:
cat_col_train = []
for col in cars_train.columns:
    if cars_train[col].dtype == 'object':
        cat_col_train.append(col)
cat_col_train

['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'classes']

In [26]:
cat_col_test = []
for col in cars_test.columns:
    if cars_test[col].dtype == 'object':
        cat_col_test.append(col)
cat_col_test

['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']

In [28]:
le = LabelEncoder()

for col in cat_col_train:
    cars_train[col] = le.fit_transform(cars_train[col])
cars_train.head()

,buying,maint,doors,persons,lug_boot,safety,classes
0,3,0,1,2,2,1,2
1,1,3,1,1,2,2,2
2,1,0,3,2,0,1,2
3,0,2,2,0,2,2,2
4,1,1,1,2,0,2,1


In [29]:
lm = LabelEncoder()

for col in cat_col_test:
    cars_test[col] = le.fit_transform(cars_test[col])
cars_test.head()

,buying,maint,doors,persons,lug_boot,safety
0,2,3,3,1,2,1
1,3,0,0,0,0,2
2,1,0,0,2,2,1
3,3,3,1,0,0,0
4,1,2,2,1,1,2


In [30]:
X = cars_train.values[:,:-1]
Y = cars_train.values[:,-1]
Y = Y.astype(int)

In [34]:
print(X.shape)
print(Y.shape)

(1382, 6)
(1382,)


In [36]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X

array([[ 1.33507272, -1.3488262 , -0.45682233,  1.21505861,  1.22565305,
         0.00176987],
       [-0.44760409,  1.32688358, -0.45682233, -0.01064285,  1.22565305,
         1.22474807],
       [-0.44760409, -1.3488262 ,  1.33418038,  1.21505861, -1.21505663,
         0.00176987],
       ...,
       [-1.33894249,  1.32688358,  1.33418038, -0.01064285,  0.00529821,
        -1.22120833],
       [ 0.44373431,  0.43498032,  0.43867903, -0.01064285, -1.21505663,
         0.00176987],
       [ 0.44373431, -0.45692294,  1.33418038,  1.21505861,  1.22565305,
        -1.22120833]])

In [38]:
x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size=0.2, random_state=10)

In [39]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(1105, 6)
(277, 6)
(1105,)
(277,)


In [42]:
model_DecisionTree = DecisionTreeClassifier(criterion='gini', random_state=10)

model_DecisionTree.fit(x_train,y_train)
y_pred = model_DecisionTree.predict(x_test)
y_pred

array([2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2,
       3, 0, 2, 0, 3, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2,
       0, 2, 0, 0, 2, 2, 0, 3, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 2,
       2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 3, 2, 2, 0, 0, 2, 2, 2, 0, 2,
       0, 3, 2, 0, 3, 2, 2, 2, 2, 2, 3, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2,
       2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 3, 1, 0, 2, 2, 0, 2, 2, 2, 2,
       2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2,
       2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 0, 0, 2, 3, 1, 2, 2, 0, 0, 2,
       0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 0, 2, 0, 0,
       0, 1, 1, 2, 2, 2, 0, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 0, 2, 2, 0, 0, 2, 2, 1, 0, 2, 2])

In [43]:
print(list(zip(y_pred,y_test)))

[(2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (1, 1), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (2, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (0, 0), (0, 1), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (3, 3), (0, 0), (2, 2), (0, 0), (3, 3), (0, 0), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (0, 1), (2, 2), (0, 0), (0, 0), (2, 2), (2, 2), (0, 0), (3, 3), (0, 0), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (3, 3), (2, 2), (2, 2), (0, 0), (0, 0), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (0, 0), (3, 3), (2, 2), (0, 0), (3, 3), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (3, 3), (2, 2), (0, 0), (2, 2), (0, 0),

In [44]:
cfm = confusion_matrix(y_test,y_pred)
print(cfm)

print("classification_report")
print(classification_report(y_test,y_pred))

acc = accuracy_score(y_test,y_pred)
print("Accuracy : ",acc)

[[ 69   1   1   0]
 [  4   8   0   0]
 [  0   0 185   0]
 [  0   0   0   9]]
classification_report
              precision    recall  f1-score   support

           0       0.95      0.97      0.96        71
           1       0.89      0.67      0.76        12
           2       0.99      1.00      1.00       185
           3       1.00      1.00      1.00         9

    accuracy                           0.98       277
   macro avg       0.96      0.91      0.93       277
weighted avg       0.98      0.98      0.98       277

Accuracy :  0.9783393501805054


In [45]:
print(list(zip(cars_train.columns[0:-1], model_DecisionTree.feature_importances_)))

[('buying', 0.21976792783843332), ('maint', 0.18220929667385857), ('doors', 0.062005934167191426), ('persons', 0.19425872158174767), ('lug_boot', 0.09772725134941933), ('safety', 0.24403086838934968)]


In [46]:
model_DecisionTree.score(x_train,y_train)

1.0

In [49]:
with open("model_DecisionTree.text","w") as f:
    f = export_graphviz(model_DecisionTree,feature_names=cars_train.columns[:-1],out_file=f)

In [59]:
model_DecisionTree = DecisionTreeClassifier(criterion='gini', random_state=10,
                                            splitter='best',min_samples_leaf=5,
                                            max_depth=10,max_leaf_nodes=10)
model_DecisionTree.fit(x_train,y_train)
y_pred = model_DecisionTree.predict(x_test)

In [60]:
print(list(zip(y_pred,y_test)))

[(2, 2), (2, 2), (2, 2), (0, 2), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (0, 1), (3, 0), (2, 2), (2, 2), (2, 2), (0, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (0, 0), (2, 2), (2, 2), (0, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (2, 0), (3, 1), (2, 2), (0, 2), (0, 2), (2, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (0, 2), (3, 3), (0, 0), (2, 2), (0, 0), (3, 3), (3, 0), (0, 2), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (0, 0), (2, 2), (2, 2), (2, 0), (2, 2), (2, 2), (0, 1), (2, 2), (0, 0), (0, 0), (2, 2), (2, 2), (0, 0), (3, 3), (0, 0), (0, 0), (2, 2), (0, 2), (0, 0), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 0), (3, 3), (2, 2), (2, 2), (0, 0), (0, 0), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (0, 0), (3, 3), (0, 2), (0, 0), (3, 3), (2, 2), (0, 2), (2, 2), (2, 2), (2, 2), (3, 3), (2, 2), (0, 0), (2, 2), (0, 0),

In [62]:
cfm = confusion_matrix(y_test,y_pred)
print(cfm)

print("classification_report")
print(classification_report(y_test,y_pred))

acc = accuracy_score(y_test,y_pred)
print("Accuracy : ",acc)

[[ 56   0   8   7]
 [  7   0   0   5]
 [ 18   0 166   1]
 [  0   0   0   9]]
classification_report
              precision    recall  f1-score   support

           0       0.69      0.79      0.74        71
           1       0.00      0.00      0.00        12
           2       0.95      0.90      0.92       185
           3       0.41      1.00      0.58         9

    accuracy                           0.83       277
   macro avg       0.51      0.67      0.56       277
weighted avg       0.83      0.83      0.83       277

Accuracy :  0.8339350180505415


c:\Users\amalp\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\amalp\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\amalp\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [63]:
with open("model_DecisionTree2.text","w") as f:
    f = export_graphviz(model_DecisionTree,feature_names=cars_train.columns[:-1],out_file=f)

In [87]:
svm = SVC(kernel='rbf', C=20, gamma=0.1)
svm.fit(x_train,y_train)
y_pred = svm.predict(x_test)

In [88]:
print(list(zip(y_test,y_pred)))

[(2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (1, 1), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (0, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (0, 0), (1, 0), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (3, 3), (0, 0), (2, 2), (0, 0), (3, 3), (0, 0), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (1, 0), (2, 2), (0, 0), (0, 0), (2, 2), (2, 2), (0, 0), (3, 3), (0, 0), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (3, 3), (2, 2), (2, 2), (0, 0), (0, 0), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (0, 0), (3, 3), (2, 2), (0, 0), (3, 3), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (3, 3), (2, 2), (0, 0), (2, 2), (0, 0),

In [89]:
cfm = confusion_matrix(y_test,y_pred)
print(cfm)

print("classification_report")
print(classification_report(y_test,y_pred))

acc = accuracy_score(y_test,y_pred)
print("Accuracy : ",acc)

[[ 68   1   2   0]
 [  4   7   0   1]
 [  1   0 184   0]
 [  0   0   0   9]]
classification_report
              precision    recall  f1-score   support

           0       0.93      0.96      0.94        71
           1       0.88      0.58      0.70        12
           2       0.99      0.99      0.99       185
           3       0.90      1.00      0.95         9

    accuracy                           0.97       277
   macro avg       0.92      0.88      0.90       277
weighted avg       0.97      0.97      0.97       277

Accuracy :  0.9675090252707581


In [106]:
KNN = KNeighborsClassifier(metric='euclidean', n_neighbors=15)
KNN.fit(x_train,y_train)
y_pred = KNN.predict(x_test)

In [107]:
print(list(zip(y_test,y_pred)))

[(2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (1, 1), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (0, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (0, 2), (1, 0), (2, 2), (2, 0), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (3, 3), (0, 0), (2, 2), (0, 0), (3, 0), (0, 0), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 2), (2, 2), (0, 2), (2, 2), (2, 2), (0, 2), (2, 2), (2, 2), (1, 0), (2, 2), (0, 0), (0, 2), (2, 2), (2, 2), (0, 0), (3, 3), (0, 0), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 2), (3, 0), (2, 2), (2, 2), (0, 0), (0, 0), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (0, 0), (3, 0), (2, 2), (0, 2), (3, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (3, 0), (2, 2), (0, 0), (2, 2), (0, 0),

In [108]:
cfm = confusion_matrix(y_test,y_pred)
print(cfm)

print("classification_report")
print(classification_report(y_test,y_pred))

acc = accuracy_score(y_test,y_pred)
print("Accuracy : ",acc)

[[ 51   0  19   1]
 [ 11   1   0   0]
 [  2   0 183   0]
 [  7   0   0   2]]
classification_report
              precision    recall  f1-score   support

           0       0.72      0.72      0.72        71
           1       1.00      0.08      0.15        12
           2       0.91      0.99      0.95       185
           3       0.67      0.22      0.33         9

    accuracy                           0.86       277
   macro avg       0.82      0.50      0.54       277
weighted avg       0.85      0.86      0.83       277

Accuracy :  0.855595667870036


In [119]:
rf = RandomForestClassifier(n_estimators=100, random_state=10)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)

In [120]:
print(list(zip(y_test,y_pred)))

[(2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (1, 1), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (0, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (0, 0), (1, 0), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (3, 3), (0, 0), (2, 2), (0, 0), (3, 3), (0, 0), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (0, 2), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (1, 0), (2, 2), (0, 0), (0, 0), (2, 2), (2, 2), (0, 0), (3, 3), (0, 0), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (0, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 0), (3, 3), (2, 2), (2, 2), (0, 0), (0, 0), (2, 2), (2, 2), (2, 2), (0, 0), (2, 2), (0, 0), (3, 3), (2, 2), (0, 0), (3, 3), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (3, 3), (2, 2), (0, 0), (2, 2), (0, 0),

In [121]:
cfm = confusion_matrix(y_test,y_pred)
print(cfm)

print("classification_report")
print(classification_report(y_test,y_pred))

acc = accuracy_score(y_test,y_pred)
print("Accuracy : ",acc)

[[ 65   1   5   0]
 [  4   8   0   0]
 [  1   0 184   0]
 [  0   0   0   9]]
classification_report
              precision    recall  f1-score   support

           0       0.93      0.92      0.92        71
           1       0.89      0.67      0.76        12
           2       0.97      0.99      0.98       185
           3       1.00      1.00      1.00         9

    accuracy                           0.96       277
   macro avg       0.95      0.89      0.92       277
weighted avg       0.96      0.96      0.96       277

Accuracy :  0.9602888086642599


In [130]:
classifier = LogisticRegression()
kfold = KFold(n_splits=10, random_state=10, shuffle=True)
kfold_cvResult = cross_val_score(estimator=classifier, X=x_train, y=y_train, cv=kfold)
print(kfold_cvResult)
print(kfold_cvResult.mean())


[0.63963964 0.72972973 0.65765766 0.69369369 0.68468468 0.65454545
 0.76363636 0.7        0.64545455 0.75454545]
0.6923587223587224


In [131]:
for train_value, test_value in kfold.split(x_train):
    classifier.fit(x_train[train_value], y_train[train_value]).predict(x_train[test_value])

In [135]:
y_pred=classifier.predict(x_test)
print(list(zip(y_test,y_pred)))

[(2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 0), (0, 2), (2, 2), (2, 2), (1, 0), (0, 0), (2, 2), (2, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 0), (0, 2), (0, 2), (2, 2), (2, 2), (2, 0), (2, 2), (2, 2), (2, 2), (2, 2), (0, 3), (2, 0), (0, 2), (1, 2), (2, 2), (2, 0), (2, 2), (0, 2), (2, 2), (2, 2), (0, 2), (2, 2), (2, 2), (2, 2), (2, 0), (2, 2), (3, 0), (0, 2), (2, 2), (0, 2), (3, 0), (0, 2), (2, 2), (0, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 2), (2, 2), (0, 2), (2, 2), (2, 0), (0, 2), (2, 0), (2, 2), (1, 2), (2, 2), (0, 2), (0, 2), (2, 2), (2, 2), (0, 0), (3, 3), (0, 2), (0, 2), (2, 2), (2, 2), (0, 2), (2, 2), (0, 2), (2, 2), (2, 2), (0, 2), (2, 2), (2, 2), (0, 2), (2, 0), (2, 2), (2, 2), (0, 2), (2, 2), (2, 2), (0, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (0, 2), (3, 0), (2, 2), (2, 2), (0, 2), (0, 2), (2, 0), (2, 2), (2, 2), (0, 2), (2, 2), (0, 2), (3, 3), (2, 2), (0, 0), (3, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (3, 2), (2, 2), (0, 2), (2, 2), (0, 0),

In [136]:
cfm = confusion_matrix(y_test,y_pred)
print(cfm)

print("classification_report")
print(classification_report(y_test,y_pred))

acc = accuracy_score(y_test,y_pred)
print("Accuracy : ",acc)

[[ 11   0  57   3]
 [  2   0  10   0]
 [ 19   0 166   0]
 [  5   0   2   2]]
classification_report
              precision    recall  f1-score   support

           0       0.30      0.15      0.20        71
           1       0.00      0.00      0.00        12
           2       0.71      0.90      0.79       185
           3       0.40      0.22      0.29         9

    accuracy                           0.65       277
   macro avg       0.35      0.32      0.32       277
weighted avg       0.56      0.65      0.59       277

Accuracy :  0.6462093862815884


c:\Users\amalp\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\amalp\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\amalp\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
